In [6]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

#Lasso regression

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

from itertools import combinations
import scipy.stats as sts
import seaborn as sns

#for decision tree & random forest
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier


In [7]:
#import dataset
data = pd.read_csv("../data/thyroidDF.csv")

In [8]:
display(data['age'])

0       29
1       29
2       41
3       36
4       32
        ..
9167    56
9168    22
9169    69
9170    47
9171    31
Name: age, Length: 9172, dtype: int64

In [11]:
#clean the data
data = data[data['age'] <= 110]
data = data[data['age']> 0]
data = data.drop_duplicates('patient_id')


,age,TSH,T3,TT4,T4U,FTI,TBG,target,patient_id,sex_F,...,FTI_measured_f,FTI_measured_t,TBG_measured_f,TBG_measured_t,referral_source_STMW,referral_source_SVHC,referral_source_SVHD,referral_source_SVI,referral_source_WEST,referral_source_other
0,29,0.3,NaN,NaN,NaN,NaN,NaN,-,840801013,1,...,1,0,1,0,0,0,0,0,0,1
1,29,1.6,1.9,128.0,NaN,NaN,NaN,-,840801014,1,...,1,0,1,0,0,0,0,0,0,1
2,41,NaN,NaN,NaN,NaN,NaN,11.0,-,840801042,1,...,1,0,0,1,0,0,0,0,0,1
3,36,NaN,NaN,NaN,NaN,NaN,26.0,-,840803046,1,...,1,0,0,1,0,0,0,0,0,1
4,32,NaN,NaN,NaN,NaN,NaN,36.0,S,840803047,1,...,1,0,0,1,0,0,0,0,0,1


,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
167,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,other,F,NaN
5256,NaN,F,f,f,f,f,f,t,f,f,...,NaN,t,NaN,t,NaN,t,NaN,other,-,NaN
6044,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,-,NaN
6045,NaN,M,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,-,NaN
6747,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,K,NaN
6773,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,-,NaN
6862,NaN,M,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,other,-,NaN
6863,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,-,NaN
6880,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,other,-,NaN
6934,NaN,F,f,f,f,f,f,f,f,f,...,NaN,t,NaN,t,NaN,t,NaN,SVI,-,NaN


In [8]:
data.head(5)

,age,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,...,target,patient_id,sex_F,sex_M,on_thyroxine_0,on_thyroxine_1,thyroid_surgery_0,thyroid_surgery_1,tumor_0,tumor_1
167,40,f,f,f,f,f,f,f,f,f,...,F,840827019,1,0,1,0,1,0,1,0
5256,35,f,f,f,f,f,f,f,f,f,...,-,851128040,1,0,1,0,0,1,1,0
6044,77,f,f,f,f,f,f,f,f,f,...,-,860305064,1,0,1,0,1,0,1,0
6045,73,f,f,f,f,f,f,f,f,f,...,-,860305065,0,1,1,0,1,0,1,0
6747,77,f,f,f,f,f,f,f,f,f,...,K,860702030,1,0,1,0,1,0,1,0


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Create a Lasso regression model
lasso = Lasso(alpha=0.1)

# Fit the model to the training data
lasso.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = lasso.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)